**Exploratory Data Analysis with Pandas**

Based on http://www.awesomestats.in/python-pandas-titanic/

Download titanic.xlsx from [here](https://www.dropbox.com/s/1tt96wl7pub14am/titanic.xlsx?dl=0) and save under `PythonTutorial/data/` on your computer

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_excel("data/titanic.xlsx")

In [ ]:
# Examine the data: print the first 3 rows
df.head(3)

In [ ]:
# the last 3 rows
df.tail(3)

In [ ]:
# How many cells and rows are there?

df.shape # (n rows, n cols)

In [ ]:
# What are the columns and their data types?
df.dtypes
# Note: 
# sibsp - number of siblings or spouses on board
# parch - number of children or parents on board

In [ ]:
# Print basic statistics
df.describe()

Survival factors
========

In [ ]:
# How many people survived?

# value_counts counts unique values in a column
df.survived.value_counts()

In [ ]:
# How many people survived, in percent?
df.survived.value_counts(normalize=True)

In [ ]:
# How many men and women survived and how many died?
pd.crosstab(df.survived, df.sex)

In [ ]:
# Percent of women among survivors? percent of men among survivors?
pd.crosstab(df.survived, df.sex, normalize="index")

In [ ]:
# Percent of survivors among women? percent of survivors among men?
pd.crosstab(df.survived, df.sex, normalize="columns")

In [ ]:
# How many children under 5 on board?
len(df[df.age <= 5])

In [ ]:
# How many of them survived?
df[df.age <= 5]["survived"].value_counts()

In [ ]:
# Alternatively:
len(df[(df.age <= 5) & (df.survived == 1)])

In [ ]:
# How many children under 5 survived, in percent?
df[df.age <= 5]["survived"].value_counts(normalize=True)

In [ ]:
# Did having a spouse or sibling on-board increase survival chances?

# What are possible values of sibsp?
df['sibsp'].value_counts()

In [ ]:
# What is the average number of siblings or spouses per survivor?
df[df['survived'] == 1]['sibsp'].mean()

In [ ]:
# What is the average number of siblings or spouses for non-survivors?
df[df['survived'] == 0]['sibsp'].mean()

In [ ]:
# Is the difference significant?
# independent t-test
from scipy.stats import ttest_ind

series1 = df[df['survived'] == 1]['sibsp']
series2 = df[df['survived'] == 0]['sibsp']

tval, pval = ttest_ind(series1.values, series2.values)

print("T test: %.5f, at p=%.5f" % (tval, pval))

In [ ]:
# Did women who had a spouse or sibling have a greater chance or survival than those who did not?

# first, create a new column which has 0 for no siblings or spouse and 1 for having at least one sibling or spouse
def sibs_func(row):
    if row['sibsp'] > 0:
        return 1
    else:
        return 0
    
df['has_sibsp'] = df.apply(sibs_func, axis=1)

In [ ]:
df.head(3)

In [ ]:
crosstab_result = pd.crosstab(df[df.sex=="female"].survived, df[df.sex=="female"].has_sibsp)
crosstab_result

In [ ]:
# Significance test: do the observed counts different from those that would be expected if the two series were independent?
from scipy.stats import chi2_contingency

chi2_val, pval, dof, exp_vals = chi2_contingency(crosstab_result)
print("Chi^2 value: %.3f, p value: %.5f" % (chi2_val, pval))

Points of Embarkation
=====================

In [ ]:
# What are all the points of embarkation?
df.embarked.unique()

In [ ]:
# How many people embarked at each point? print also the number of those whose point of embarkation is unknown.
df.embarked.value_counts(dropna=False)

Destinations
======

In [ ]:
# What were the 10 most common final destinations?

df["home.dest"].value_counts().head(10)

In [ ]:
# What are the most common states which were the final destinations?

# create a new column with the destination state
df['state.dest'] = df['home.dest'].str.extract("([A-Z][A-Z])", expand=False)
df

In [ ]:
# What are the 10 most common values in the state.dest column?
df['state.dest'].value_counts(dropna=False).head(10)

In [ ]:
# How many people were headed for Canada?

# create a temporary df so that we can drop some rows from it
df2 = df.copy()
df2 = df2.dropna(subset=["home.dest"])

# search based on regular expressions
df2[df2["home.dest"].str.contains("Canada|ON|PQ|BC")]

In [ ]:
# How many people were heading to Canada?
len(df2[df2["home.dest"].str.contains("Canada|ON|PQ|BC")])

Passenger class
===============

In [ ]:
# How many people were there whose age is unknown?
len(df[df.age.isnull()])

In [ ]:
# Clean the data: drop all records where at least one column value is undefined
# df = df.dropna()

In [ ]:
# Clean the data: drop all records where either age or pclass is undefined
df = df.dropna(subset=['age','pclass'], how='any')

In [ ]:
# How many records are left?
df.shape

In [ ]:
# What is the mean age in each passenger class?
df.groupby('pclass')['age'].mean()

In [ ]:
# histogram of ages of passengers in 1st class
df[df.pclass==1]['age'].hist(bins=10)

In [ ]:
# histogram of ages of passengers in 3rd class
df[df.pclass==3]['age'].hist(bins=10)

In [ ]:
# What was the mean age of different sexes in each passenger class?
df.groupby(['pclass', 'sex'])['age'].mean()

In [ ]:
# How many survivors in each class, percent-wise?
df.groupby('pclass').survived.value_counts(normalize=True)